In [3]:
import numpy as np
from header import *
from nsp.utils.base_conv import *
import argparse
import sys
sys.path.append('../nsp')
from nsp.utils.func import *
from nsp.utils.local2global import *

/home/user/miniconda3/envs/project/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [105]:
E4 = np.load("/home/user/project/doc/data/shastry_exact_J1=1.npy")

In [109]:
T = 1
beta = 1/T
np.sum(E4 * np.exp(-beta*E4))/np.sum(np.exp(-beta*E4))

-5.585519722674071

## prepare local ham

In [110]:
u = np.array([
    [0,1,0,0],
    [1/np.sqrt(2), 0, 1/np.sqrt(2), 0],
    [-1/np.sqrt(2), 0, 1/np.sqrt(2), 0],
    [0,0,0,1]
])

U = np.kron(u, u)

In [122]:
J = [0.35, 1] #J, J_D

In [123]:
Sz = np.zeros([2,2])
Sz[0,0] = 1/2
Sz[1,1] = -1/2
Sx = np.zeros([2,2])
Sx[1,0] = 1/2
Sx[0,1] = 1/2
Sy = np.zeros([2,2], dtype=np.complex64)
Sy[1,0] = 1j/2
Sy[0,1] = -1j/2

SzSz = np.kron(Sz,Sz).real.astype(np.float64)
SxSx = np.kron(Sx,Sx).real.astype(np.float64)
SySy = np.kron(Sy,Sy).real.astype(np.float64)

lh = SzSz + SxSx + SySy
lh = -lh

In [124]:
H1 = sum_ham(J[0]*lh, [[1,2],[1,3]], 4, 2)
H1 += sum_ham(J[1]*lh/4, [[0,1],[2,3]], 4, 2)

H2 = sum_ham(J[0]*lh, [[0,2],[0,3]], 4, 2)
H2 += sum_ham(J[1]*lh/4, [[0,1],[2,3]], 4, 2)

E1 = np.linalg.eigvalsh(H1)
E2 = np.linalg.eigvalsh(H2)

In [125]:
H1, H2 = U.T@H1@U, U.T@H2@U
print(H1)

[[ 3.75000000e-01  0.00000000e+00  6.93889390e-18  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00 -5.55111512e-18
   6.93889390e-18  0.00000000e+00  6.93889390e-18  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.25000000e-01  0.00000000e+00  0.00000000e+00
  -3.93551224e-18  0.00000000e+00 -1.75000000e-01  0.00000000e+00
   0.00000000e+00  1.75000000e-01  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.54074396e-33  0.00000000e+00  1.25000000e-01  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.75000000e-01
   1.54074396e-33  0.00000000e+00 -1.54074396e-33  0.00000000e+00
   0.00000000e+00  1.75000000e-01  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.25000000e-01
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.75000000e-01
   0.00

In [126]:
E1

array([-0.3      , -0.3      , -0.3      , -0.3      , -0.3      ,
       -0.1628123, -0.1628123, -0.1628123,  0.125    ,  0.125    ,
        0.125    ,  0.225    ,  0.3378123,  0.3378123,  0.3378123,
        0.375    ])

### prepair plaquette 

In [ ]:
square1 = [[0,1], [1,2], [2,3], [3,0]]
square2 = [[4,5], [5,6], [6,7], [7,4]]
H1 = sum_ham(J[0]*lh/4, square1 + square2, 8, 2)
H1 += sum_ham(J[1]*lh, [[1,7]], 4, 2)

H2 = sum_ham(J[0]*lh/4, square1 + square2, 8, 2)
H2 += sum_ham(J[1]*lh, [[2,4]], 4, 2)

In [98]:
os.getcwd()

'/home/user/project/python/exact/SS'

In [100]:
H1 = np.load("../../array/SS/plq_original_J=[1.0, 1]/0.npy")

In [104]:
(2**4)**4

65536

In [102]:
print(H1.shape)

(256, 256)


## optimize unitary

### dimer basis

In [53]:
D = 4

models = nsp.model.UnitaryRiemanGenerator(D, dtype=torch.float64)

In [54]:
loss_f = nsp.loss.MES
from datetime import datetime
from random import randint

In [62]:
D = 4
M = 10
loss1 = loss_f(H1, [D, D], pout = False)
loss2 = loss_f(H2, [D, D], pout = False)
best_fun = 1E10

for _ in range(M):
    seed = randint(0, 2<<32 - 1)
    torch.manual_seed(seed)
    np.random.seed(seed)
#     models = [nsp.model.UnitaryRiemanGenerator(D, dtype=torch.float64) for _ in range(2)]
    model = nsp.model.UnitaryRiemanGenerator(D, dtype=torch.float64)
    cg = RiemanNonTransUnitaryCG([(model, model)]*2, [loss1, loss2], pout = False)
    solver = UnitaryNonTransTs(cg, af=False)
    ret = solver.run(2000, False)
    print(f"res = {ret.fun} / seed = {seed}")
    if ret.fun < best_fun:
        print(f"\nbest_fun updated : {ret.fun}\n")
        best_fun = ret.fun
        best_model = ret.model
# LHs = [loss._transform_kron([best_model[i].matrix(), best_model[(i+1)%L].matrix()], original=True).detach().numpy() for i in range(L)]
# folder = f"../../array/FL/optm_nt{L}Jp={(J[0]):.2}_Jx={(J[1]):.2}_{ham_type}_{loss_name}"
# co = nsp.utils.base_conv.change_order

target loss      : 1.3000000119
initial loss     : 1.7798828929
loss upper bound : 1.5000000119




  1%|██▉                                                                                                                                                                                                                         | 27/2000 [00:00<00:23, 83.77it/s, iter=26, loss=1.30000]


No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
stack in local minimum --> break loop
res = 1.3000000119208908 / seed = 3046045476

best_fun updated : 1.3000000119208908

target loss      : 1.3000000119
initial loss     : 1.9955372456
loss upper bound : 1.5000000119




  2%|████▋                                                                                                                                                                                                                       | 43/2000 [00:00<00:29, 67.35it/s, iter=42, loss=1.50000]


No local minimum found
stack in local minimum --> break loop
res = 1.5000000119206234 / seed = 3222352951
target loss      : 1.3000000119
initial loss     : 1.7897753081
loss upper bound : 1.5000000119




...

No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
stack in local minimum --> break loop
res = 1.3000000119210653 / seed = 2851178001
target loss      : 1.3000000119
initial loss     : 2.0629344310
loss upper bound : 1.5000000119




  3%|██████▌                                                                                                                                                                                                                     | 60/2000 [00:00<00:29, 65.37it/s, iter=59, loss=1.77918]


stack in local minimum --> break loop
res = 1.7791814384438325 / seed = 1859769114
target loss      : 1.3000000119
initial loss     : 1.9053715810
loss upper bound : 1.5000000119




  1%|██▉                                                                                                                                                                                                                         | 27/2000 [00:00<00:21, 90.16it/s, iter=26, loss=1.30000]

No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
No local minimum found
stack in local minimum --> break loop
res = 1.3000000119213626 / seed = 2233820119


### optimzie for many J

In [2]:
import re

def process_line(stdout):
    for line in str(stdout).split("\\n"):
        pat = "best fun is : "
        if pat in line:
            line = line.replace(pat,"")
            return float(line)
            


In [87]:
print(f"{0.123:.2}")

0.12


In [5]:
J = np.arange(0.1, 1.1, 0.01)

In [6]:
J

array([0.1 , 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 ,
       0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31,
       0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42,
       0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53,
       0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64,
       0.65, 0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75,
       0.76, 0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86,
       0.87, 0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97,
       0.98, 0.99, 1.  , 1.01, 1.02, 1.03, 1.04, 1.05, 1.06, 1.07, 1.08,
       1.09])

In [12]:
stdout

b'save matrix ((16, 16)):  ../../array/SS/dimer_basis_J_[0.1, 1]/0.npy\nsave matrix ((16, 16)):  ../../array/SS/dimer_basis_J_[0.1, 1]/1.npy\n'

In [15]:
import numpy as np
import os 
os.chdir("/home/user/project/python/exact/SS")
import subprocess
energy_array = []
sign_array = []
time_array = []
for J in np.arange(0.1, 1.1, 0.01):
    out = subprocess.Popen(["python", "make_ss_local.py", "-l", "dimer_optm" ,"-J", f"{J:.2}", "-M", "100"], 
               stdout=subprocess.PIPE, 
               stderr=subprocess.STDOUT)
    stdout,stderr = out.communicate()

    best = process_line(stdout)
    print(f"best fun at J = {J:.2} is : ",best)
#     energy_array.append(energy)
#     sign_array.append(sign)
#     time_array.append(time)

best fun at J = 0.1 is :  1.09999999403114
best fun at J = 0.11 is :  1.1100000142911228
best fun at J = 0.12 is :  1.120000004755787
best fun at J = 0.13 is :  1.1299999952225246
best fun at J = 0.14 is :  1.1399999856831173
best fun at J = 0.15 is :  1.1500000059552327
best fun at J = 0.16 is :  1.15999999641438
best fun at J = 0.17 is :  1.1699999868751367
best fun at J = 0.18 is :  1.180000007144566
best fun at J = 0.19 is :  1.1899999976046256
best fun at J = 0.2 is :  1.1999999880750118
best fun at J = 0.21 is :  1.2100000083331413
best fun at J = 0.22 is :  1.219999998795069
best fun at J = 0.23 is :  1.2299999892637108
best fun at J = 0.24 is :  1.2400000095210872
best fun at J = 0.25 is :  1.249999999986863
best fun at J = 0.26 is :  1.2599999904579366
best fun at J = 0.27 is :  1.2699999809159264
best fun at J = 0.28 is :  1.279999971381255
best fun at J = 0.29 is :  1.2900000214468847
best fun at J = 0.3 is :  1.30000001191368
best fun at J = 0.31 is :  1.310000002372305
bes